In [8]:
// wsl
// conda activate cling
// jupyter notebook
// copy and paste url

In [9]:
#include <iostream>
#include <stdio.h>
using namespace std;

In [10]:

class Vector {
    public:
        float x = 0;
        float y = 0;
    
        Vector(float _x = 0, float _y = 0){
            x = _x;
            y = _y;
        }

        // Operator overloads.

        // Addition increment operator
		Vector& operator +=(const Vector& vector) {                           
			x = x + vector.x;
			y = y + vector.y;
			return *this;
		}

		// Addition operator
		friend Vector operator +(Vector vectorA, const Vector& vectorB) {
			vectorA += vectorB;
			return vectorA;
		}

		// Addition operator
		Vector& operator *=(const float scalar) {
			x *= scalar;
			y *= scalar;
			return *this;
		}

		// Addition operator
		friend Vector operator *(Vector vector, const float scalar) {
			vector *= scalar;
			return vector;
		}

		string ToString() {
			return "(" + to_string(x).substr(0, 4) + ", " + to_string(y).substr(0, 4) + ")";
		}

};

In [32]:
class  Joint {
    public:
        string name;
        float m_Length;
        float m_Angle;
        Joint* m_Children[16];
        int m_ChildrenCount = 0;
        Vector m_Vertices[4096];
        
        Joint(string _name = "", float length = 0, float angle = 0) {
            name = _name;
            m_Length = length;
            m_Angle = angle;
        };

        // Operates on the assumption that we aren't going to add more than 4 children.
        Joint* AddChild(string _name, float length, float angle) {
            m_Children[m_ChildrenCount] = new Joint{ _name, length, angle };
            m_ChildrenCount++;
            return m_Children[m_ChildrenCount - 1];
        }

        Joint* AddChild(string _name, Joint joint) {
            // Creates a new variable at this address.
            m_Children[m_ChildrenCount] = new Joint{ };
            // Copies the values of the passed in local variable to the appropriate address.
            Joint* newJoint = m_Children[m_ChildrenCount];
            memcpy(newJoint, &joint, sizeof(joint));
            newJoint->name = _name;
            m_ChildrenCount++;
            return m_Children[m_ChildrenCount - 1];
        }

        void PrintHierarchy(int depth = 0) {
            for (int i = 0; i < depth + 1; i++) {
                cout << "-->";
            }
            cout << "  " << name << " : " << to_string(m_Length) << " : " << to_string(m_Angle) << "\n\n";
            for (int i = 0; i < m_ChildrenCount; i++) {
                m_Children[i]->PrintHierarchy(depth + 1);
            }
        }
    
    void UpdateVertices(Vector vertices[], int* vertexCount, Vector origin = Vector{}) {
            
            cout << *vertexCount;
            Vector vertex = GetVertex(origin);
            vertices[*vertexCount] = vertex;
            (*vertexCount)++;
            for (int i = 0; i < m_ChildrenCount; i++) {
                m_Children[i]->UpdateVertices(vertices, vertexCount, vertex);
            }
        }

        Vector GetVertex(Vector origin) {
            float angle = m_Angle / 180 * M_PI;
            Vector vertex = origin + Vector{ m_Length * cos(angle), m_Length * sin(angle) };
            cout << "  " << name << " : " << vertex.ToString() << "\n\n";
            return vertex;
        }
};

In [33]:
class Skeleton {
    public:
        static Joint Shoulder(bool isMirrored = false) {
            // Flip should
            string suffix = ".l";
            if (isMirrored) { 
                suffix = ".r"; 
            }
            Joint shoulder = Joint{ "shoulder" + suffix, 1, (float)isMirrored * 180 };

            Joint* elbow = shoulder.AddChild("elbow" + suffix, 1, (float)isMirrored * 180);
            (*elbow).AddChild("wrist" + suffix, 1, (float)isMirrored * 180);

            return shoulder;
        }

        static Joint Spine() {
            Joint spine = Joint{ "spine", 1, 90 };
            Joint* torso = spine.AddChild("torso", 1, 90);
            Joint* neck = torso->AddChild("neck", 1, 90);
            Joint shoulder_l = Skeleton::Shoulder();
            Joint shoulder_r = Skeleton::Shoulder(true);
            neck->AddChild("shoulder_l", shoulder_l);
            neck->AddChild("shoulder_r", shoulder_r);
            return spine;
        }
};

In [34]:
Joint spine = Skeleton::Spine();
spine.PrintHierarchy();
int vertexCount = 0;
spine.UpdateVertices(spine.m_Vertices, &vertexCount);


-->  spine : 1.000000 : 90.000000

-->-->  torso : 1.000000 : 90.000000

-->-->-->  neck : 1.000000 : 90.000000

-->-->-->-->  shoulder_l : 1.000000 : 0.000000

-->-->-->-->-->  elbow.l : 1.000000 : 0.000000

-->-->-->-->-->-->  wrist.l : 1.000000 : 0.000000

-->-->-->-->  shoulder_r : 1.000000 : 180.000000

-->-->-->-->-->  elbow.r : 1.000000 : 180.000000

-->-->-->-->-->-->  wrist.r : 1.000000 : 180.000000

0  spine : (-0.0, 1.00)

1  torso : (-0.0, 2.00)

2  neck : (-0.0, 3.00)

3  shoulder_l : (1.00, 3.00)

4  elbow.l : (2.00, 3.00)

5  wrist.l : (3.00, 3.00)

6  shoulder_r : (-1.0, 3.00)

7  elbow.r : (-2.0, 3.00)

8  wrist.r : (-3.0, 3.00)



In [7]:
cout << 1;

1

In [36]:
cout << sizeof(spine) * 2048;

67469312